The most important key building block in `LangChain` is the `chain`.
A `chain` usually combines an LLM with a prompt, but also allows us to combine other chains and perform several operations on text.
One of the powers of a `chain` is running it over many inputs at a time.

# `LLMChain`

The `LLMChain` is a simple, but powerful chain that underpins a lot of the chains covered in this lesson.

In [1]:
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
api_version = "2023-12-01-preview"
model = "gpt-35-turbo-16k"

We initialize our model with a high temperature to elicit more randomness ("creativity") in the response.

In [3]:
chat = AzureChatOpenAI(model=model, temperature=0.9, api_version=api_version)

In [4]:
prompt = ChatPromptTemplate.from_template(
    template="What is the best name to describe a company that makes {product}?"
)

We combine the prompt and the model using the `LLMChain`.

In [5]:
chain = LLMChain(prompt=prompt, llm=chat)

We can run the chain by providing an input to the `run` method.

In [6]:
product = "Queen Size Sheet Set"
chain.run(product)

'RoyalRest'

# `SimpleSequentialChain`

The `LLMChain` is the most simple, basic type of chain in `LangChain`.
The `SimpleSequentialChain` runs a sequence of chains one after another.
This works well when we have chains that expect one input and return one output.

In [7]:
from langchain.chains import SimpleSequentialChain

To demonstrate, we'll treat our prior `LLMChain` as our first chain in a sequence of chains.

In [8]:
chain_one = chain.copy()

Now we create a second chain to take the output of the first chain (`chain_one`), a company name, and pass it into a second chain.

In [9]:
second_prompt = ChatPromptTemplate.from_template(
    template="Write a 20 word description for the following company: {company_name}"
)

chain_two = LLMChain(prompt=second_prompt, llm=chat)

We supply both chains to `SimpleSequentialChain` in the order we want to run.

In [10]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

We run the `simple_chain` like any other chain using the `run` method.
Remember that we are inputting a product as that is what the first chain (`chain_one`) expects.

In [11]:
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


AttributeError: 'LLMChain' object has no attribute 'callbacks'

Nearly 200 versions of `LangChain` have come out since the the DeepLearningAI tutorial was created.
After some toying I found that I needed to supply `output_key="company_name"` to `chain_one`.

In [12]:
chain_one = LLMChain(prompt=prompt, llm=chat, output_key="company_name")

This doesn't affect the output.

In [13]:
chain_one.run(product)

'"Royalty Linens" or "Regal Comforts"'

In [14]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
RoyalRest Beddings
RoyalRest Beddings is a luxury bedding company that offers premium mattresses, pillows, and linens for a comfortable and rejuvenating sleep experience.

> Finished chain.


'RoyalRest Beddings is a luxury bedding company that offers premium mattresses, pillows, and linens for a comfortable and rejuvenating sleep experience.'

# `SequentialChain`

The `SimpleSequentialChain` works well when there is a single input and single output.
To supply more than one input or return more than one output we use the `SequentialChain`.

In [15]:
from langchain.chains import SequentialChain

We generate an intial chain to translate a review into English.

In [16]:
prompt_one = ChatPromptTemplate.from_template(
    template="Translate the following review to english:\n\n{review}"
)
chain_one = LLMChain(prompt=prompt_one, llm=chat, output_key="english_review")

We generate a second chain that summarizes the output of `chain_one`.

In [17]:
prompt_two = ChatPromptTemplate.from_template(
    template="Can you summarize the following reveiw in 1 sentence:\n\n{english_review}"
)
chain_two = LLMChain(prompt=prompt_two, llm=chat, output_key="summary")

We generate a third chain that determines what language the original review is in, using the same input as `chain_one`.

In [18]:
prompt_three = ChatPromptTemplate.from_template(
    template="What language is the following review:\n\n{review}"
)
chain_three = LLMChain(prompt=prompt_three, llm=chat, output_key="language")

We generate a fourth chain that responds to the summary produced by `chain_two` in the language produced by `chain_three`.

In [19]:
prompt_four = ChatPromptTemplate.from_template(
    template="""Write a follow up response to the following summary in the specified language:
    
    Summary: {summary}
    
    Language: {language}"""
)
chain_four = LLMChain(prompt=prompt_four, llm=chat, output_key="followup_message")

Finally, we generate a final chain that combines `chain_one`, `chain_two`, `chain_three`, and `chain_four`.

In [20]:
final_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "followup_message"],
    verbose=True,
)

In [21]:
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"
final_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer expresses disappointment with the mediocre taste of the product, highlighting issues with the foam not holding, implying a possible difference in quality compared to the same product purchased in stores.',
 'followup_message': "Cher critique,\n\nNous vous remercions d'avoir partagé votre avis concernant notre produit. Nous sommes sincèrement désolés d'apprendre que vous avez été déçu par son goût moyen ainsi que par les problèmes rencontrés avec la tenue de la mousse. Nous accordons une grande importance à la qualité de nos produits et nous sommes soucieux de satisfaire nos clients.\n\nNous souhaiterions avoir plus 

It's important to note that the `input_keys` and `output_keys` need to be precise.

# `LLMRouterChain`

The `LLMRouterChain` allows us to route inputs to a chain, respective of the input.
For example, we can have multiple sub-chains, each specialized for a specific input, handled via routing.

In [22]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

Suppose we are routing inputs on four different subjects.

In [23]:
physics_template = """You are a very smart physics professor.
You are great at answering questions about physics in a concise
and easy to understand manner.
When you don't know the answer to a question you admit
that you don't know.

Here is a queston:
{input}"""

In [24]:
math_template = """You are a very good mathematician.
You are great at answering math questions.
You are so good because you are able to break down
hard problems into their component parts,
answer the component parts, and then put them together
to answer the broader question.

Here is the question:
{input}"""

In [25]:
history_template = """You are a very good historian.
You have an excellent knowledge of and understanding of people,
events and contexts from a range of historical periods.
You have the ability to think, reflect, debate, discuss and
evaluate the past. You have a resepect for historical evidence
and the ability to make use of it to support your explanations
and judgments.

Here is a question:
{input}"""

In [26]:
computer_science_template = """You are a successful computer scientist.
You have a passion for creativity, collaboration,
forward-thinking, confidence, strong problem-solving capabilities,
understanding of theories and algorithms, and excellent communication
skills. You are great at answering coding questions.
You are so good because you know how to solve a problem by
describing the solution in imperative steps
that a machine can easily interpret and you know how to
choose a solution that has a good balance between
time complexity and space complexity.

Here is the question:
{input}"""

We can add more information about our prompts which we will pass to a `MultiPromptChain`.

In [27]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering questions about math",
        "prompt_template": math_template,
    },
    {
        "name": "history",
        "description": "Good for answering questions about history",
        "prompt_template": history_template,
    },
    {
        "name": "computer_science",
        "description": "Good for answering questions about computer science",
        "prompt_template": computer_science_template,
    }
]

In [28]:
# Lowering temperature to reduce randomness.
chat = AzureChatOpenAI(model=model, temperature=0, api_version=api_version)

In [29]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=template)
    chain = LLMChain(prompt=prompt, llm=chat)
    destination_chains[name] = chain

Each `chain` in `destination_chains` will be called by the `LLMRouterChain`.

In [30]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In addition to the `destination_chains`, we need a default chain that will be called when the `LLMRouterChain` can't decided which sub-chain to use.

In [31]:
default_prompt = ChatPromptTemplate.from_template(template="{input}")
default_chain = LLMChain(prompt=default_prompt, llm=chat)

We define a prompt to instruct the model on how to route between the different chains.
There are also general instructions for what needs to be done, as well as what the output format should be.

In [32]:
multi_prompt = """Given a raw text input to a
langauge model, select the model prompt best suited for the input.
You will be given the names of the available prompts and a
description of what the prompt is best suited for.
You may also revise the original input if you think that revising
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt
names specified below OR it can be "DEFAULT" if the input is not
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json) >>"""

To put it all together, we first format the `multi_prompt` with the `destinations_str`.

In [33]:
router_template = multi_prompt.format(destinations=destinations_str)

We create the prompt using `PromptTemplate`.
We use `PromptTemplate` because it provides more paramters than the `ChatPromptTemplate` we've been using.
Note that the `RouterOutputParser` is important as it will help the chain decided which subchain to route between.

In [34]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

We combine the `router_prompt` with the model into a `LLMRouterChain` instance.

In [35]:
router_chain = LLMRouterChain.from_llm(llm=chat, prompt=router_prompt)

Finally, we put it all together to create the final chain.

In [36]:
final_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [37]:
final_chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


C:\Users\A2644752\Sandbox\PRIVATE-RAI\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it is an idealized object that absorbs all radiation without any reflection or transmission. \n\nAccording to Planck\'s law, the intensity and spectrum of black body radiation depend solely on the temperature of the object. As the temperature increases, the peak wavelength of the radiation shifts towards shorter wavelengths, resulting in a change in color. For example, a black body at room temperature emits mostly infrared radiation, while a hotter black body, like a red-hot iron, emits visible light.\n\nThe concept of black body radiation played a crucial role in the development of quantum mechanics. Max Planck introduced the idea of quantization to explain the observed spectrum of black body radiation, which led to the birth of quantum theory. Albert Einstein further expanded on this concept with his e

In [38]:
final_chain.run("What is the area under the function y=x^2 from -1 to +1?")



> Entering new MultiPromptChain chain...


C:\Users\A2644752\Sandbox\PRIVATE-RAI\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


math: {'input': 'What is the area under the function y=x^2 from -1 to +1?'}
> Finished chain.


"Thank you for your kind words! I'll be happy to help you with your question.\n\nTo find the area under the function y = x^2 from -1 to +1, we need to calculate the definite integral of the function over that interval.\n\nThe definite integral of a function represents the signed area between the function and the x-axis over a given interval. In this case, we want to find the area between the curve y = x^2 and the x-axis from x = -1 to x = 1.\n\nTo solve this, we can use the definite integral formula:\n\n∫[a to b] f(x) dx\n\nIn our case, a = -1 and b = 1, and f(x) = x^2. So, the integral becomes:\n\n∫[-1 to 1] x^2 dx\n\nTo evaluate this integral, we can use the power rule for integration. According to the power rule, the integral of x^n with respect to x is (x^(n+1))/(n+1), where n is any real number except -1.\n\nApplying the power rule to our integral, we get:\n\n∫[-1 to 1] x^2 dx = [(x^3)/3] evaluated from -1 to 1\n\nEvaluating the integral at the limits, we have:\n\n[(1^3)/3] - [(-1

In [39]:
final_chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...


C:\Users\A2644752\Sandbox\PRIVATE-RAI\env\Lib\site-packages\langchain\chains\llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

The last example failed, but I was impressed by the math example!
Rather than using prompt engineering to handle situations like this, I'd use `Pydantic` and some kind of retry method.